# Neural Network

In [ ]:
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.neural_network import MLPClassifier
import pandas as pd
import seaborn as sns


In [ ]:
train = pd.read_csv('data/training.csv')



In [ ]:
train = train.sample(frac=.01)

In [ ]:
train['Label'].value_counts()

In [ ]:
len(train.columns)

In [ ]:
# 50 inputs, 6 outputs
model = MLPClassifier(hidden_layer_sizes=[30, 12])
model.fig()